<a href="https://colab.research.google.com/github/hahachang/colab/blob/main/BioAssignMission.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [38]:
countParallelMachine = 4

In [1]:
def StartdayParseStock():
  from datetime import datetime, timedelta
  if (datetime.now()+timedelta(hours=8)).strftime("%H%M%S")<"160000":
    date   = (datetime.now()+timedelta(hours=8)+timedelta(days=-1)).strftime("%Y%m%d")  
  else:
    date   = (datetime.now()+timedelta(hours=8)).strftime("%Y%m%d")
  return date

global dateStartday
dateStartday =  StartdayParseStock()
print("======  " +  dateStartday  + "  ======"  )
#dateStartday="20210514"

======  20210521  ======


In [8]:
def ParseTWSE_OHLC(date):
  #date format: str and continue number
  #example    : "20210514"

  print("ParseTWSE_OHLC lanuching ..")
  from urllib import request
  import time
  import json
  def download_json(target_url):
      opener = request.build_opener()
      opener.addheaders = [('user-agent', 'Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/78.0.3904.108 Safari/537.36')]
      request.install_opener(opener) 
      with request.urlopen(target_url) as url:
          data = json.loads(url.read().decode())
      return data
  
  #if (datetime.now()+timedelta(hours=8)).strftime("%H%M%S")<"160000":
  #  date   = (datetime.now()+timedelta(hours=8)+timedelta(days=-1)).strftime("%Y%m%d")  
  #else:
  #  date   = (datetime.now()+timedelta(hours=8)).strftime("%Y%m%d")
  
  timestamp = 1000*int(time.mktime(time.localtime()))
  url_TWSE_OHLC = "https://www.twse.com.tw/exchangeReport/MI_INDEX?response=json&date="+date+"&type=ALL&_="+str(timestamp)
  json_TWSE_OHLC = download_json(url_TWSE_OHLC)
  print("ParseTWSE_OHLC closing ..")
  return json_TWSE_OHLC


#def FilterStockTWSE_OHLC(jf):


def LoopsTWSE_OHLC(jf):
  import pandas as pd
  df_raw_all_stock_price = pd.DataFrame(jf['data9'],columns=jf['fields9'])

  #包含所有的權證及股票
  df_all_stock_price = df_raw_all_stock_price.replace(',','', regex=True) 
  df_all_stock_price= df_all_stock_price.replace(',','', regex=True)
  #去除comma

  #去除所有權證
  rule1 = [str(x).zfill(6) for x in range(30001,90000)] 
      #認購權證之代號為6個阿拉伯數字，編號從030001至089999。
  rule2 = "P|F|Q|C|B|X|Y"
      #認售權證之代號為5個阿拉伯數字後加上英文字「P」，編號從03001P至08999P。
      #以外國交易所之指數或有價證券作為標的之認購權證代號為5個阿拉伯數字後加上英文字「F」，編號從03001F至08999F。
      #以外國交易所之指數或有價證券作為標的之認售權證代號為5個阿拉伯數字後加上英文字「Q」，編號從03001Q至08999Q。
      #牛證之代號為5個阿拉伯數字後加上英文字「C」，編號從03001C至08999C。
      #熊證之代號為5個阿拉伯數字後加上英文字「B」，編號從03001B至08999B。
      #展延型牛證之代號為5個阿拉伯數字後加上英文字「X」，編號從03001X至08999X。
      #展延型熊證之代號為5個阿拉伯數字後加上英文字「Y」，編號從03001Y至08999Y。
      
  df_stock_price = df_all_stock_price[~df_all_stock_price.證券代號.isin(rule1)] 
  df_stock_price = df_stock_price[~df_stock_price.證券代號.str.contains(rule2)]

  df_stock_price.成交金額 = df_stock_price.成交金額.astype(float)
  loops = list(df_stock_price.sort_values(by="成交金額",ascending=False)["證券代號"].values)
  return loops

In [ ]:
loops = LoopsTWSE_OHLC(ParseTWSE_OHLC(dateStartday))
loops

In [14]:
def partition(lst, n):
    division = len(lst) / n
    return [lst[round(division * i):round(division * (i + 1))] for i in range(n)]

In [39]:
missions = partition(loops,countParallelMachine)

In [46]:
def SendFileToDropbox(fileLocal,fileDropbox):
    ##### upload file to dropbox #####

    try:
        import dropbox
        #print("module is installed")
    except ModuleNotFoundError:
        print("module is not installed")
        # or
        !pip install dropbox
        import dropbox

    from dropbox.files import WriteMode
    from dropbox.exceptions import ApiError, AuthError
    global tokenDropbox
    tokenDropbox = "9VjbJCirSqsAAAAAAAA1JuBzv7mujcQGbebgTGu2IjDup-N77QjVY8MEsg1ccizx"
    dbx = dropbox.Dropbox(tokenDropbox)
    with open(fileLocal, 'rb') as f:
      dbx.files_upload(f.read(), "/"+fileDropbox, mode=WriteMode('overwrite')) 
      f.close()

def SendProgressToDropbox(nowProgress,machine):
  import pandas as pd
  fileProgress = "bio\\"+str(dateStartday)+"\\"+"bio_progress" + str(machine) + ".csv" 
  pd.DataFrame(nowProgress).to_csv("\\"+fileProgress,index= False,header=False)
  SendFileToDropbox("\\"+fileProgress,"bio/"+str(dateStartday)+"/"+"bio_progress"+str(machine)+".csv")
  print("Send Progress"+ str(machine) + " to Dropbox ..")

In [47]:
for i in range(0,len(missions)):
  print(i)
  SendProgressToDropbox(missions[i],i)

0
Send Progress0 to Dropbox ..
1
Send Progress1 to Dropbox ..
2
Send Progress2 to Dropbox ..
3
Send Progress3 to Dropbox ..
